In [1]:
!pip install modelscope ipywidgets tqdm accelerate

Looking in indexes: https://mirrors.aliyun.com/pypi/simple
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 561.5/561.5 kB 63.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 14.9 MB/s eta 0:00:00

[notice] A new release of pip is available: 24.2 -> 25.2
[notice] To update, run: python3.11 -m pip install --upgrade pip


In [2]:
from model_download import download_model

download_model()

开始下载 Qwen-Image 模型到: ./models/Qwen-Image


2025-08-21 01:26:50,529 - modelscope - INFO - Got 25 files, start to download ...


Processing 25 items:   0%|          | 0.00/25.0 [00:00<?, ?it/s]

2025-08-21 01:34:37,807 - modelscope - INFO - Download model 'Qwen/Qwen-Image' successfully.


模型下载完成，保存在: ./models/Qwen-Image


In [3]:
!pip install ./diffusers

Looking in indexes: https://mirrors.aliyun.com/pypi/simple
Processing ./diffusers
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 799.9/799.9 kB 7.5 MB/s eta 0:00:00
  Created wheel for diffusers: filename=diffusers-0.36.0.dev0-py3-none-any.whl size=4161451 sha256=5f7d831fc4a9128b4a4a7a9fa036d74727d433843e97cc722c25043390cf399f
  Stored in directory: /tmp/pip-ephem-wheel-cache-ofotvar_/wheels/88/b3/7b/ace66df42ba9cb229e9df4095adb2d9bba30d69abc79961a1f
Successfully built diffusers

[notice] A new release of pip is available: 24.2 -> 25.2
[notice] To update, run: python3.11 -m pip install --upgrade pip


In [4]:
!pip install transformers

Looking in indexes: https://mirrors.aliyun.com/pypi/simple
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 17.9 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 17.1 MB/s eta 0:00:00

[notice] A new release of pip is available: 24.2 -> 25.2
[notice] To update, run: python3.11 -m pip install --upgrade pip


In [5]:
from importlib.metadata import version
print("diffusers version:", version("diffusers"))
print("torch version:", version("torch"))
print("transformers version:", version("transformers"))

diffusers version: 0.36.0.dev0
torch version: 2.5.1+cu124
transformers version: 4.55.2


In [6]:
import torch

# Load the pipeline
if torch.cuda.is_available():
    torch_dtype = torch.bfloat16
    device = "cuda"
else:
    torch_dtype = torch.float32
    device = "cpu"
print(f"使用的是{device}")

使用的是cuda


In [7]:
from diffusers import QwenImagePipeline

model_name = "./models/Qwen-Image"
pipe = QwenImagePipeline.from_pretrained(model_name, torch_dtype=torch_dtype)

Loading pipeline components...:   0%|          | 0/5 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

The config attributes {'pooled_projection_dim': 768} were passed to QwenImageTransformer2DModel, but are not expected and will be ignored. Please verify your config.json configuration file.


Loading checkpoint shards:   0%|          | 0/9 [00:00<?, ?it/s]

In [8]:
# from diffusers import DiffusionPipeline


# model_name = "./models/Qwen-Image"


# pipe = DiffusionPipeline.from_pretrained(model_name, torch_dtype=torch_dtype)
# print(f"管道已加载成功{pipe}")

In [9]:
pipe = pipe.to(device)

positive_magic = {
    "en": ", Ultra HD, 4K, cinematic composition.", # for english prompt
    "zh": ", 超清，4K，电影级构图." # for chinese prompt
}

# 定义5个测试提示词
test_prompts = [
    {
        "name": "cyberpunk_city",
        "prompt": '''A photo of a cyberpunk city at night, with neon lights reflecting on the rain-slicked streets. A ramen stall's sign clearly reads "未来拉面" (Future Ramen), with smaller text below saying "AI-Powered Noodles." A nearby electronic billboard scrolls with binary code: "01010100 01101111 01101110 01100111 01111001 01101001".''',
        "description": "测试霓虹灯效果下的中英文文本渲染，以及复杂背景下数字/代码的清晰度"
    },
    {
        "name": "ancient_book",
        "prompt": '''A close-up shot of a yellowed, ancient book open on a heavy wooden desk, with a quill pen beside it. On the page, beautiful calligraphy reads "靜以修身，儉以養德" (Cultivate oneself in tranquility, nurture virtue through thrift). At the bottom of the page, a faded English note says: "The virtues are cultivated in tranquility".''',
        "description": "测试不同中文书法风格和手写斜体英文的渲染，以及文本在羊皮纸等古老材质上的纹理"
    },
    {
        "name": "product_packaging",
        "prompt": '''A high-end, close-up shot of a modern, minimalist skincare product box. The front of the box features the product name "光子精华" (Photon Essence) and the brand "Quantum Glow." The ingredient list is clearly visible: "Aqua, Glycerin, Peptide-5, Hyaluronic Acid (玻尿酸)".''',
        "description": "测试现代产品设计上印刷文本的清晰度和纹理，评估模型处理中英混合排版、大小写敏感性和技术术语的能力"
    },
    {
        "name": "street_graffiti",
        "prompt": '''A weathered brick wall in a city alley, covered in colorful graffiti art. At the center of the graffiti is a large Chinese character "龙" (Dragon), with "DRAGON'S BREATH" written next to it in a spray-paint style font. In the corner, a small signature reads: "By Artist Qwen, 2024".''',
        "description": "测试非标准艺术字体（如喷漆和手写）的渲染，以及文本与砖墙粗糙纹理的融合效果"
    },
    {
        "name": "code_screen",
        "prompt": '''A programmer's desk with a computer monitor displaying code. A Python function is clearly visible on the screen: "def calculate_fibonacci(n): # 计算斐波那契数列\n    if n <= 1:\n        return n". A sticky note with the handwritten text "Launch Deadline: 12/25" is faintly visible in the screen's reflection.''',
        "description": "测试等宽字体（如代码字体）的准确渲染，包括缩进、语法高亮和双语注释，以及屏幕反射等复杂光学效果下的详细文本处理"
    }
]

negative_prompt = " " # Recommended if you don't use a negative prompt.

# Generate with different aspect ratios
aspect_ratios = {
    "1:1": (1328, 1328),
    "16:9": (1664, 928),
    "9:16": (928, 1664),
    "4:3": (1472, 1104),
    "3:4": (1104, 1472),
    "3:2": (1584, 1056),
    "2:3": (1056, 1584),
}

width, height = aspect_ratios["16:9"]

# 依次生成5张图片
for i, test_case in enumerate(test_prompts, 1):
    print(f"\n正在生成第{i}张图片: {test_case['name']}")
    print(f"测试目标: {test_case['description']}")
    print(f"提示词: {test_case['prompt'][:100]}...")
    
    image = pipe(
        prompt=test_case['prompt'] + positive_magic["en"],
        negative_prompt=negative_prompt,
        width=width,
        height=height,
        num_inference_steps=50,
        true_cfg_scale=4.0,
        generator=torch.Generator(device="cuda").manual_seed(42 + i)  # 每张图片使用不同的种子
    ).images[0]
    
    filename = f"{test_case['name']}_test.png"
    image.save(filename)
    print(f"图片已保存为: {filename}")
    
print("\n所有5张测试图片生成完成！")

  0%|          | 0/50 [00:00<?, ?it/s]